In [1]:
# !pip install python-rake

In [2]:
# !pip install summa

In [3]:
# !pip install spacy-udpipe

In [4]:
import RAKE
import nltk
from nltk.corpus import stopwords
import pandas as pd 
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
import collections
import spacy_udpipe
spacy_udpipe.download("ru")

/Users/tbkazakova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/tbkazakova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/tbkazakova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/tbkazakova/anaconda3/lib/python3.7/site-packages/tensorflow/

Already downloaded a model for the 'ru' language


In [5]:
stop = stopwords.words('russian')
stop.extend(['который', 'оно', 'весь', 'это', 'оба', 'ещё', 'свой', 'очень', 'затем', 'каков', 'наш', 'вкупе', 'например',
             'возможно', 'увы', 'примерно', 'некоторый', 'кстати', 'именно', 'лишь', 'вроде', 'вовсе', 'каждый', 'поскольку',
             'вполне', 'либо', 'нея', 'поэтому', '—', 'довольно', 'столь', 'что'])

In [6]:
# Лемматизация текстов
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

### Задание 1

Миникорпус состоит из 9 новостей с сайта http://astro.uni-altai.ru/.

После текста новости даются ключевые слова. Написано: "Ключевые слова: ..."

### Задание 2

Я разметила ключевые слова. Эталоном стало объединение двух списков.

В файликах с текстами перед содержанием есть абзац с ключевыми словами от сайта и с эталонными (=их+мои)

In [7]:
# Получили корпус из файликов
all_kws = []
all_texts = []
for i in range(9):
    filename = 'HW1_keywords_corpus/' + str(i+1) + '.txt'
    with open(filename, encoding='utf8') as f:
        info = f.readlines()
        kw = info[2].strip().split(': ')[1].split(', ')
        text = info[4:]
        text = ''.join(text)
    all_kws.append(kw)
    all_texts.append(text)

In [8]:
# Нормализуем кс так, как мы их будем получать от методов
all_kws_norm = []
for kws in all_kws:
    all_kws_norm.append(list(map(normalize_text, kws)))
all_kws_norm

[['галактика',
  'тёмный материя',
  'тёмный вещество',
  'sdss',
  'карликовый галактика',
  'спутниковый галактика',
  'спутник',
  'карлик',
  'млечный путь',
  'василий белокуров',
  'слоановский цифровой обзор небо'],
 ['планет',
  'экзопланет',
  'субар',
  'кек',
  'hires',
  'hat-p-7b',
  'ретроградный орбита',
  'обратный орбита',
  'вращение'],
 ['планет',
  'экзопланет',
  'ядро',
  'обитаемый зона',
  'планетарный ядро',
  'магнитный пол',
  'обитание'],
 ['земля',
  'полярный сияние',
  'магнитный полюс',
  'сибирь',
  'аляска',
  'движение',
  'канада'],
 ['сатурн',
  'cassini',
  'полярный сияние',
  'aurora',
  'uvis',
  'кассинь',
  'сияние'],
 ['ракета', 'солнечный парус', 'institute of space', 's-310-34', 'космос'],
 ['солнечный система',
  'галактика',
  'млечный путь',
  'квазар',
  'параллакс',
  'w3oh',
  'спиральный рукав',
  'тригонометрический параллакс',
  'расстояние'],
 ['hubble', 'галактика', 'vlt', 'квазар', 'чёрный дыра'],
 ['солнечный система',
  'астро

### Задание 3
#### Rake

In [9]:
rake = RAKE.Rake(stop)

У меня всего два "ключевых слова" длиннее двух слов, поэтому было решено сделать maxWords=2, чтобы много лишних сочетаний не попадало.

In [10]:
all_rake_kw = []
for i in range(9):
    rake_kw_list = []
    rake_kw_res = rake.run(normalize_text(all_texts[i]), maxWords=2, minFrequency=1)
    for res in rake_kw_res:
        if res[1] > 1.5:
            rake_kw_list.append(res[0])
    all_rake_kw.append(rake_kw_list)
all_rake_kw

[['оставаться незамеченный',
  'несколько сотня',
  'таинственный материя',
  'обнаруживаться исключительно',
  'гравитационный взаимодействие',
  'обычный вещество',
  'светов луч',
  'относиться непосредственно',
  'точный классификация',
  'должный содержаться',
  'участок небо',
  'окружающий область',
  'соответствующий статья',
  'astrophysical journal',
  'имя созвездие',
  'волос вероника',
  'левый iv',
  'четвёрка находиться',
  'самый мелкий',
  'подобно большинство',
  'скорее хоббит',
  'шутить белокуров',
  'русский перевод',
  'исследователь смочь',
  'маленькая спутник',
  'туманность андромеда',
  'умудриться заглотить',
  'протогалактический фрагмент',
  'самый известный',
  'левый i',
  'левый ii',
  'малый медведица',
  'изображение m',
  'daniel zucker',
  'речь идти',
  'белокуров назвать',
  'отбросить окончание',
  'содержаться большой',
  'большой пёс',
  'большой часть',
  'общий количество',
  'данные sdss',
  'точнее оценить',
  'удаться найти',
  'гораздо с

#### TextRank

In [11]:
all_trank_kw = []
for i in range(9):
    trank_kw_list = []
    trank_kw_res = keywords.keywords(normalize_text(all_texts[i]), language='russian', additional_stopwords=stop, scores=True)
    for res in trank_kw_res:
        if res[1] > 0.1:
            trank_kw_list.append(res[0])
    all_trank_kw.append(trank_kw_list)
all_trank_kw

[['год',
  'обнаружить',
  'sdss',
  'карликовый галактика найти хоббит астроном',
  'большой',
  'новое спутник',
  'галактический',
  'млечный путь',
  'самый',
  'скопление',
  'спутников'],
 ['орбита',
  'вращение',
  'наблюдение',
  'учёный',
  'американский',
  'японский',
  'hat',
  'of',
  'телескоп',
  'the',
  'звезда',
  'специалист',
  'arxiv',
  'превосходить',
  'транзитный',
  'наклон',
  'период чрезвычайно',
  'результат',
  'использоваться',
  'использовать',
  'ося',
  'ось'],
 ['ядро мочь',
  'планета',
  'планет',
  'мир',
  'зона',
  'формирование',
  'образ',
  'сигера',
  'пока',
  'создаваться',
  'создавать',
  'различный',
  'успеть',
  'условие',
  'ряд',
  'жизнь далеко',
  'мантия',
  'жидкий вода'],
 ['северный',
  'магнитный полюс',
  'год',
  'мочь',
  'учёный',
  'нормальный',
  'государственный',
  'меняться',
  'изучение образцов',
  'заявить джозеф',
  'американский геофизический'],
 ['сияние',
  'солнечный',
  'снимка',
  'ультрафиолетовый диапазон

#### Tfidf

In [12]:
all_texts_norm = []
for i in range(9):
    all_texts_norm.append(normalize_text(all_texts[i]))

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(all_texts_norm)
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
#df

In [13]:
all_tfidf_kw = []
df_dict = df.to_dict()
keys = list(df_dict)
for i in range(9):
    tfidf_kw = []
    dict_word_n = collections.Counter()
    for key in keys:
        dict_word_n[key] = df_dict[key][i]
    for res in dict_word_n.most_common(10):
        if res[1] > 0.153:
            if res[0] not in stop:
                tfidf_kw.append(res[0])
    all_tfidf_kw.append(tfidf_kw)
all_tfidf_kw

[['галактика', 'карлик', 'карликовый', 'год', 'млечный', 'обнаружить', 'sdss'],
 ['вращение', '7b', 'hat', 'орбита'],
 ['ядро', 'жизнь', 'планета', 'мир'],
 ['магнитный', 'полюс', 'северный', 'сибирь', 'год', 'последний'],
 ['снимка', 'сатурн', 'сделать', 'сияние'],
 ['парус', 'аппарат', 'солнечный', 'космос', 'развернуть'],
 ['расстояние', 'параллакс', 'рукав', 'метод', 'галактика', 'измерить'],
 ['квазар', 'галактика', '2958', 'he0450', 'дыра', 'чёрный'],
 ['пояс',
  'друг',
  'койперо',
  'объект',
  'компонент',
  'гравитационный',
  'масса',
  'пара']]

### Задание 4
#### Шаблоны

In [14]:
# all_kws_norm
# У нас в кс NOUN, ADJ+NOUN, ADJ+VERB, X, X+NOUN. (Не рассматриваем кс>2слов.)
shablon = [['NOUN'], ['ADJ', 'NOUN'], ['ADJ', 'VERB'], ['X'], ['X', 'NOUN']]

In [15]:
nlp = spacy_udpipe.load("ru")

In [16]:
all_sorted_rake_kw_pos = []
all_sorted_trank_kw_pos = []
all_sorted_tfidf_kw_pos = []

for i in range(9):
    sorted_rake_kw_pos = []
    sorted_trank_kw_pos = []
    sorted_tfidf_kw_pos = []
    
    for kw in all_rake_kw[i]:
        doc = nlp(kw)
        rake_kw_pos = []
        for token in doc:
            rake_kw_pos.append(token.pos_)
        if rake_kw_pos in shablon:
            sorted_rake_kw_pos.append([kw, rake_kw_pos])
    all_sorted_rake_kw_pos.append(sorted_rake_kw_pos)
    
    for kw in all_trank_kw[i]:
        doc = nlp(kw)
        trank_kw_pos = []
        for token in doc:
            trank_kw_pos.append(token.pos_)
        if trank_kw_pos in shablon:
            sorted_trank_kw_pos.append([kw, trank_kw_pos])
    all_sorted_trank_kw_pos.append(sorted_trank_kw_pos)
    
    for kw in all_tfidf_kw[i]:
        doc = nlp(kw)
        tfidf_kw_pos = []
        for token in doc:
            tfidf_kw_pos.append(token.pos_)
        if tfidf_kw_pos in shablon:
            sorted_tfidf_kw_pos.append([kw, tfidf_kw_pos])
    all_sorted_tfidf_kw_pos.append(sorted_tfidf_kw_pos)

In [17]:
# Убираем из эталонных несколько кс, не подходящих по шаблону (давайте считать, что мы их и изначально не выбирали)
all_sorted_kws_norm_pos = []

for i in range(9):
    sorted_kws_norm_pos = []
    
    for kw in all_kws_norm[i]:
        doc = nlp(kw)
        kws_norm_pos = []
        for token in doc:
            kws_norm_pos.append(token.pos_)
        if kws_norm_pos in shablon:
            sorted_kws_norm_pos.append([kw, kws_norm_pos])
    all_sorted_kws_norm_pos.append(sorted_kws_norm_pos)

### Задание 5

Будем оценивать, что получилось...

In [18]:
print('Результаты Rake, TextRank, TfIDf для каждого текста (без шаблонов)\n')
for i in range(9):
    print('Текст' + str(i+1) + '\t точность\t полнота\t Fмера')
    
    rake_intersect = 0
    trank_intersect = 0
    tfidf_intersect = 0
    
    for rake in all_rake_kw[i]:
        if rake in all_kws_norm[i]:
            rake_intersect += 1
    rake_precision = rake_intersect / len(all_rake_kw[i])
    rake_recall = rake_intersect / len(all_kws_norm[i])
    if rake_intersect == 0:
        rake_fm = 0
    else:
        rake_fm = (2 * rake_precision * rake_recall) / (rake_precision + rake_recall)
    print('Rake\t', rake_precision, '\t', rake_recall, '\t', rake_fm)
    
    for trank in all_trank_kw[i]:
        if trank in all_kws_norm[i]:
            trank_intersect += 1
    trank_precision = trank_intersect / len(all_trank_kw[i])
    trank_recall = trank_intersect / len(all_kws_norm[i])
    if trank_intersect == 0:
        trank_fm = 0
    else:
        trank_fm = (2 * trank_precision * trank_recall) / (trank_precision + trank_recall)
    print('TRank\t', trank_precision, '\t', trank_recall, '\t', trank_fm)
    
    for tfidf in all_tfidf_kw[i]:
        if tfidf in all_kws_norm[i]:
            tfidf_intersect += 1
    tfidf_precision = tfidf_intersect / len(all_tfidf_kw[i])
    tfidf_recall = tfidf_intersect / len(all_kws_norm[i])
    if tfidf_intersect == 0:
        tfidf_fm = 0
    else:
        tfidf_fm = (2 * tfidf_precision * tfidf_recall) / (tfidf_precision + tfidf_recall)
    print('TfIDf\t', tfidf_precision, '\t', tfidf_recall, '\t', tfidf_fm, '\n')

Результаты Rake, TextRank, TfIDf для каждого текста (без шаблонов)

Текст1	 точность	 полнота	 Fмера
Rake	 0.037037037037037035 	 0.18181818181818182 	 0.061538461538461535
TRank	 0.18181818181818182 	 0.18181818181818182 	 0.18181818181818182
TfIDf	 0.42857142857142855 	 0.2727272727272727 	 0.33333333333333326 

Текст2	 точность	 полнота	 Fмера
Rake	 0.14285714285714285 	 0.1111111111111111 	 0.125
TRank	 0.045454545454545456 	 0.1111111111111111 	 0.06451612903225805
TfIDf	 0.25 	 0.1111111111111111 	 0.15384615384615383 

Текст3	 точность	 полнота	 Fмера
Rake	 0.05405405405405406 	 0.2857142857142857 	 0.09090909090909091
TRank	 0.05555555555555555 	 0.14285714285714285 	 0.08
TfIDf	 0.25 	 0.14285714285714285 	 0.18181818181818182 

Текст4	 точность	 полнота	 Fмера
Rake	 0.06666666666666667 	 0.14285714285714285 	 0.09090909090909091
TRank	 0.09090909090909091 	 0.14285714285714285 	 0.1111111111111111
TfIDf	 0.16666666666666666 	 0.14285714285714285 	 0.15384615384615383 

Текст5

In [19]:
print('Результаты Rake, TextRank, TfIDf для каждого текста (c шаблонами)\n')
for i in range(9):
    print('Текст' + str(i+1) + '\t точность\t полнота\t Fмера')
    
    rake_intersect = 0
    trank_intersect = 0
    tfidf_intersect = 0
    
    for rake in all_sorted_rake_kw_pos[i]:
        if rake in all_sorted_kws_norm_pos[i]:
            rake_intersect += 1
    rake_precision = rake_intersect / len(all_sorted_rake_kw_pos[i])
    rake_recall = rake_intersect / len(all_sorted_kws_norm_pos[i])
    if rake_intersect == 0:
        rake_fm = 0
    else:
        rake_fm = (2 * rake_precision * rake_recall) / (rake_precision + rake_recall)
    print('Rake\t', rake_precision, '\t', rake_recall, '\t', rake_fm)
    
    for trank in all_sorted_trank_kw_pos[i]:
        if trank in all_sorted_kws_norm_pos[i]:
            trank_intersect += 1
    trank_precision = trank_intersect / len(all_sorted_trank_kw_pos[i])
    trank_recall = trank_intersect / len(all_sorted_kws_norm_pos[i])
    if trank_intersect == 0:
        trank_fm = 0
    else:
        trank_fm = (2 * trank_precision * trank_recall) / (trank_precision + trank_recall)
    print('TRank\t', trank_precision, '\t', trank_recall, '\t', trank_fm)
    
    for tfidf in all_sorted_tfidf_kw_pos[i]:
        if tfidf in all_sorted_kws_norm_pos[i]:
            tfidf_intersect += 1
    tfidf_precision = tfidf_intersect / len(all_sorted_tfidf_kw_pos[i])
    tfidf_recall = tfidf_intersect / len(all_sorted_kws_norm_pos[i])
    if tfidf_intersect == 0:
        tfidf_fm = 0
    else:
        tfidf_fm = (2 * tfidf_precision * tfidf_recall) / (tfidf_precision + tfidf_recall)
    print('TfIDf\t', tfidf_precision, '\t', tfidf_recall, '\t', tfidf_fm, '\n')

Результаты Rake, TextRank, TfIDf для каждого текста (c шаблонами)

Текст1	 точность	 полнота	 Fмера
Rake	 0.125 	 0.2222222222222222 	 0.16
TRank	 0.3333333333333333 	 0.2222222222222222 	 0.26666666666666666
TfIDf	 0.75 	 0.3333333333333333 	 0.46153846153846156 

Текст2	 точность	 полнота	 Fмера
Rake	 0.5 	 0.1111111111111111 	 0.1818181818181818
TRank	 0.08333333333333333 	 0.1111111111111111 	 0.09523809523809525
TfIDf	 0.25 	 0.1111111111111111 	 0.15384615384615383 

Текст3	 точность	 полнота	 Fмера
Rake	 0.08333333333333333 	 0.16666666666666666 	 0.1111111111111111
TRank	 0.09090909090909091 	 0.16666666666666666 	 0.11764705882352942
TfIDf	 0.25 	 0.16666666666666666 	 0.2 

Текст4	 точность	 полнота	 Fмера
Rake	 0.16666666666666666 	 0.14285714285714285 	 0.15384615384615383
TRank	 0.3333333333333333 	 0.14285714285714285 	 0.2
TfIDf	 0.5 	 0.14285714285714285 	 0.22222222222222224 

Текст5	 точность	 полнота	 Fмера
Rake	 0.0 	 0.0 	 0
TRank	 0.42857142857142855 	 0.428571428

### Задание 6

Я выбрала (из любопытства и желания почитать что-то астрономическое) очень специфические тексты об исследованиях космоса, поэтому не сильно удивлена, что так много ошибок и такие низкие показатели.

Конечно, с шаблонами точность улучшилась, потому что мы откинули заведомо неправильные ключевые сочетания. Убрали глаголы, сочетания, которые редко бывают кс и т.д.

Без шаблонов совсем грустно получилось. Можно было бы ещё понастраивать параметры (попробовать обрезать список кс в других местах), я обрезала по последнее найденное кс для одного из девяти текстов.

Лучше всего почти везде себя проявил TfIDf, но серьёзным минусом способа является то, что на выходе получались только односложные кс. Один из простых и самых действенных способов улучшения показателей: сделать TfIDf не только для отдельных слов, но и для биграмм. А ещё этому методу немного помешало, что кс в текстах иногда пересекались и метод не чувствовал из-за этого текстовой уникальности словосочетания.

Чуть хуже получились результаты у TextRake, ещё хуже у Rake. Возможно, можно было создавать список-эталон немного по-другому, тогда бы результаты получились бы лучше. Как их работу улучшить на моих текстах - поизменять, сколько обрезать, потерять часть правильно найденных кс, но улучшить точность для остальных.

Совсем ничего не нашлось для 9ого текста. Проблема в том, что кс не повторяются много раз. В итоге нашлось много других слов, а нужные кс не нашлись. TfIDf был бы близок, если бы были биграммы.

TextRake и Rake дают слишком простые слова и словосочетания, в то время как кс у меня - слова специальные, специфические. Можно было бы к ним добавить фильтр на "особенность" (который есть у TfIDf, особенно, если сравнивать не только с другими моими текстами, но и с чем-нибудь попроще).